<a href="https://colab.research.google.com/github/MrSimple07/MachineLearning_Practice/blob/main/KMU_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Roberta

In [ ]:
from transformers import AutoModelForQuestionAnswering, AutoTokenizer, pipeline

model_name = "deepset/roberta-base-squad2"

# a) Get predictions
nlp = pipeline('question-answering', model=model_name, tokenizer=model_name)
QA_input = {
    'question': 'Why is model conversion important?',
    'context': 'The option to convert models between FARM and transformers gives freedom to the user and let people easily switch between frameworks.'
}
res = nlp(QA_input)

# b) Load model & tokenizer
model = AutoModelForQuestionAnswering.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/496M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/79.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

In [ ]:
from sklearn.metrics import f1_score

# prompts = [
#     {'question': 'Продолжение этого фильма Евгения Матвеева называется "Судьба".', 'context': 'Любовь земная'},
#     {'question': 'С ЭТИМ ЧЕЛОВЕКОМ, 33-м потомком Нина, Державин сравнивает богатого и ленивого вельможу.', 'context': 'Сарданапал (Нин, как и Сарданапал, — легендарный правитель Ассирии)'},
#     {'question': 'По мнению древнего мудреца Рюхейб-бин-Верда: "Мудрость состоит из 10 частей. Одна часть — знания и опыт." А что составляет остальные девять?', 'context': 'Молчание'},
#     {'question': 'По одной из версий, начало названия ЭТОЙ ОРГАНИЗАЦИИ происходит от греческого слова, обозначающего круг.', 'context': 'Ку-клукс-клан (греч. "киклос")'},
#     {'question': 'На девятерной вистуют только эти 2 категории людей', 'context': 'Попы и студенты (попы от жадности, а студенты — от бедности :-))'},
#     {'question': 'Эпиграф к книге "Корпорация М.И.Ф." Асприна звучит так: "Главное — не злоупотреблять...". Чем?', 'context': 'Эпиграфами'},
#     {'question': 'Кроме известных исторических полотен, ЕГО кисти принадлежат росписи "первой версии" Храма Христа Спасителя.', 'context': 'Василий Суриков'},
#     {'question': 'Именно столько морей омывает берега Италии.', 'context': 'Пять (Ионическое, Тирренское, Лигурийское, Адриатическое и Средиземное)'},
#     {'question': 'Именно ему Эразм Роттердамский посвятил свою книгу "Похвала глупости", а церковь канонизировала его в 1935 году, хотя он и написал книгу о "месте, которого нет".', 'context': 'Томас Мор ("Утопия")'},
#     {'question': 'ЭТОТ ОРЕХ является до такой степени важной частью экспорта Гренады, что даже попал на ее флаг.', 'context': 'Мускатный'},
#     {'question': 'О детях он говорил: "В их возрасте я умел рисовать, как Рафаэль, но всю жизнь учился рисовать, как они".', 'context': 'Пикассо'},
#     {'question': 'При звуках его песни "с полей уносится печаль".', 'context': 'Соловей (российский, славный птах)'},
#     {'question': '"Отыщи всему начало, и ты многое поймешь". Эта мудрость принадлежит ИМЕННО ЭТОМУ МЫСЛИТЕЛЮ.', 'context': 'Козьма Прутков'},
#     {'question': 'Самый средний в этом мире человек раскрасит мир в этот цвет.', 'context': 'Черно-белый'},
#     {'question': 'Кисти именно ЭТОГО АВТОРА принадлежит написанная в 1870 году картина "Птицелов", находящаяся в Третьяковке.', 'context': 'Василий Перов'},
#     {'question': 'ОН писал: "О живописи в супрематизме не может быть и речи. Живопись давно изжита, и сам художник — предрассудок прошлого".', 'context': 'Казимир Малевич'},
#     {'question': 'Это первый народ, о котором рассказывают и археологические находки, и летописи. 1700 лет назад они заселили степи у реки Селенги.', 'context': 'Племена Хунну (гуннов принимать)'},
#     {'question': 'Вернувшись в родной Сновск, в феврале 1918 года он организовал крупный отряд с хвастливым названием "Первая революционная армия".', 'context': '(Николай) Щорс'},
#     {'question': 'После того, что "силы утроил и резкость настроил", почему-то пропал он.', 'context': 'Цвет'},
#     {'question': 'ЭТИ РАКООБРАЗНЫЕ, живущие преимущественно на мелководье, напоминают плоды растения семейства буковых.', 'context': 'Морские желуди'},
#     {'question': 'Название ЭТОГО фильма 1981 года вынесено в заголовок статьи, посвященной фильму "Бен Гур".', 'context': 'Спорт и американский кинематограф'},
#     {'question': 'ЭТИМ СЛОВОМ называлась статья о товарищеском матче, в котором футбольная сборная Украины разгромила практически не оказавшую сопротивление сборную Эстонии.', 'context': 'Плодовая тема'},
#     {'question': '"Не ждать конца, в часы уставив взгляд — тогда и на краю" дышишь именно так.', 'context': 'Песни из альбома "У ломбарда"'},
#     {'question': 'ТАК ИСПОЛЬЗОВАЛА богиня Фрея кабана Хильдисвини.', 'context': 'Вымышленные свиньи'},
#     {'question': 'С итальянского этот термин переводится буквально как "исчезнувший как дым".', 'context': 'ДЫМ'}
# ]

prompts = [
    "Вы участвуете в викторине 'Что? Где? Когда?'. Внимательно прочитайте вопрос из категории 'Любовь' и ответьте на него. Обратите внимание на формат ответа: если он не указан в вопросе, то запишите только слово или фразу, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно.\nВопрос: Продолжение этого фильма Евгения Матвеева называется 'Судьба'.",
    "Прочитайте вопрос из категории 'Любовь' и ответьте на него: Продолжение этого фильма Евгения Матвеева называется 'Судьба'.",
    "Участвуете в викторине 'Что? Где? Когда?'. Внимание! Прочтите вопрос из раздела 'Любовь' и предоставьте ответ. Если формат ответа не указан в вопросе, запишите только слово или фразу, отвечающую на вопрос, и не предоставляйте дополнительные рассуждения.\nВопрос: Продолжение фильма Евгения Матвеева называется 'Судьба'.",
    "Внимание! Вас пригласили поучаствовать в викторине 'Что? Где? Когда?'. Ознакомьтесь с вопросом из категории 'Любовь' и предоставьте ответ. Если формат ответа не указан, запишите только слово или фразу, отвечающую на вопрос, без дополнительных рассуждений.\nВопрос: Продолжение фильма Евгения Матвеева называется 'Судьба'.",
    "Участвуете в викторине 'Что? Где? Когда?'. Ознакомьтесь с вопросом из категории 'Любовь' и предоставьте ответ. Если формат ответа не указан в вопросе, запишите только слово или фразу, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно.\nВопрос: Продолжение фильма Евгения Матвеева называется 'Судьба'.",
]

correct_predictions = 0


correct_predictions = 0
total_prompts = len(prompts)

for prompt in prompts:
    response = nlp(prompt)
    predicted_answer = response['answer']

    expected_answer = prompt['context']
    if predicted_answer == expected_answer:
        correct_predictions += 1

accuracy = correct_predictions / total_prompts

true_answers = [prompt['context'] for prompt in prompts]

predicted_answers = [nlp(prompt['question'], prompt['context'])['answer'] for prompt in prompts]

binary_true_answers = [1 if true == pred else 0 for true, pred in zip(true_answers, predicted_answers)]

f1 = f1_score(binary_true_answers, [1] * len(binary_true_answers))

print(f"F1 Score: {f1}")

print(f"Accuracy: {accuracy}")

NameError: name 'nlp' is not defined

2. MGPT

In [ ]:
from transformers import pipeline

pipe = pipeline("text-generation", model="ai-forever/mGPT")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/738 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/3.45G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.46k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.89M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.20M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/606 [00:00<?, ?B/s]

In [ ]:
temperature = 0

prompts = [
    "Вы участвуете в викторине 'Что? Где? Когда?'. Внимательно прочитайте вопрос из категории 'Любовь' и ответьте на него. Обратите внимание на формат ответа: если он не указан в вопросе, то запишите только слово или фразу, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно.\nВопрос: Продолжение этого фильма Евгения Матвеева называется 'Судьба'.",
    "Прочитайте вопрос из категории 'Любовь' и ответьте на него: Продолжение этого фильма Евгения Матвеева называется 'Судьба'.",
    "Участвуете в викторине 'Что? Где? Когда?'. Внимание! Прочтите вопрос из раздела 'Любовь' и предоставьте ответ. Если формат ответа не указан в вопросе, запишите только слово или фразу, отвечающую на вопрос, и не предоставляйте дополнительные рассуждения.\nВопрос: Продолжение фильма Евгения Матвеева называется 'Судьба'.",
    "Внимание! Вас пригласили поучаствовать в викторине 'Что? Где? Когда?'. Ознакомьтесь с вопросом из категории 'Любовь' и предоставьте ответ. Если формат ответа не указан, запишите только слово или фразу, отвечающую на вопрос, без дополнительных рассуждений.\nВопрос: Продолжение фильма Евгения Матвеева называется 'Судьба'.",
    "Участвуете в викторине 'Что? Где? Когда?'. Ознакомьтесь с вопросом из категории 'Любовь' и предоставьте ответ. Если формат ответа не указан в вопросе, запишите только слово или фразу, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно.\nВопрос: Продолжение фильма Евгения Матвеева называется 'Судьба'.",
]

for i, prompt in enumerate(prompts, start=1):
    generated_text = pipe(prompt, temperature=temperature, max_length=150)[0]['generated_text']

    print(f"\nGenerated Text - Prompt {i}:\n{generated_text}\n")



Generated Text - Prompt 1:
Вы участвуете в викторине 'Что? Где? Когда?'. Внимательно прочитайте вопрос из категории 'Любовь' и ответьте на него. Обратите внимание на формат ответа: если он не указан в вопросе, то запишите только слово или фразу, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно.
Вопрос: Продолжение этого фильма Евгения Матвеева называется 'Судьба'. Как вы думаете, почему?
Ответ: В фильме Евгения Матвеева 'Судьба' есть эпизод, в котором он рассказывает о своей жизни. В этом эпизоде


Generated Text - Prompt 2:
Прочитайте вопрос из категории 'Любовь' и ответьте на него: Продолжение этого фильма Евгения Матвеева называется 'Судьба'.
Вот и я, как и многие, не могу понять, почему я не могу смотреть этот фильм.
Я не могу понять, почему я не могу смотреть этот фильм.
Я не могу понять, почему я не могу смотреть этот фильм.
Я не могу понять, почему я не могу смотреть этот фильм.
Я не могу понять, почему я не могу смотреть этот фильм.
Я не могу понять,

3. Yi

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="01-ai/Yi-6B")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/2.18G [00:00<?, ?B/s]

4. Mistral

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="mistralai/Mistral-7B-v0.1")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/25.1k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.94G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

5. TinyLlama

In [ ]:
# Use a pipeline as a high-level helper
from transformers import pipeline

pipe = pipeline("text-generation", model="TinyLlama/TinyLlama-1.1B-Chat-v1.0")

prompts = [
    "Вы участвуете в викторине 'Что? Где? Когда?'. Внимательно прочитайте вопрос из категории 'Любовь' и ответьте на него. Обратите внимание на формат ответа: если он не указан в вопросе, то запишите только слово или фразу, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно.\nВопрос: Продолжение этого фильма Евгения Матвеева называется 'Судьба'.",
    "Прочитайте вопрос из категории 'Любовь' и ответьте на него: Продолжение этого фильма Евгения Матвеева называется 'Судьба'.",
    "Участвуете в викторине 'Что? Где? Когда?'. Внимание! Прочтите вопрос из раздела 'Любовь' и предоставьте ответ. Если формат ответа не указан в вопросе, запишите только слово или фразу, отвечающую на вопрос, и не предоставляйте дополнительные рассуждения.\nВопрос: Продолжение фильма Евгения Матвеева называется 'Судьба'.",
    "Внимание! Вас пригласили поучаствовать в викторине 'Что? Где? Когда?'. Ознакомьтесь с вопросом из категории 'Любовь' и предоставьте ответ. Если формат ответа не указан, запишите только слово или фразу, отвечающую на вопрос, без дополнительных рассуждений.\nВопрос: Продолжение фильма Евгения Матвеева называется 'Судьба'.",
    "Участвуете в викторине 'Что? Где? Когда?'. Ознакомьтесь с вопросом из категории 'Любовь' и предоставьте ответ. Если формат ответа не указан в вопросе, запишите только слово или фразу, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно.\nВопрос: Продолжение фильма Евгения Матвеева называется 'Судьба'.",
]

for i, prompt in enumerate(prompts, start=1):
    generated_text = pipe(prompt, temperature=temperature, max_length=150)[0]['generated_text']

    print(f"\nGenerated Text - Prompt {i}:\n{generated_text}\n")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:381: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(



Generated Text - Prompt 1:
Вы участвуете в викторине 'Что? Где? Когда?'. Внимательно прочитайте вопрос из категории 'Любовь' и ответьте на него. Обратите внимание на формат ответа: если он не указан в вопросе, то запишите только слово или фразу, отвечающую на вопрос, никаких дополнительных рассуждений приводить не нужно.
Вопрос: Продолжение этого фильма Евгения Матвеева называется 'Судьба'.
Ответ: «Судьба


Generated Text - Prompt 2:
Прочитайте вопрос из категории 'Любовь' и ответьте на него: Продолжение этого фильма Евгения Матвеева называется 'Судьба'.


Generated Text - Prompt 3:
Участвуете в викторине 'Что? Где? Когда?'. Внимание! Прочтите вопрос из раздела 'Любовь' и предоставьте ответ. Если формат ответа не указан в вопросе, запишите только слово или фразу, отвечающую на вопрос, и не предоставляйте дополнительные рассуждения.
Вопрос: Продолжение фильма Евгения Матвеева называется 'Судьба'. В каком году был снят фильм?
Внимание! Прочтите ответ из


Generated Text - Prompt 4:
Вним